<a href="https://colab.research.google.com/github/aarchi-dot/deep-learning/blob/main/DL_lab4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Problem Statement 1: Architecture Design Focus

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
import os

save_path = "/content/drive/MyDrive/cnn_models"
os.makedirs(save_path, exist_ok=True)

print("Models will be saved to:", save_path)


Models will be saved to: /content/drive/MyDrive/cnn_models


PROBLEM STATEMENT 2
# Imbalanced Dataset Handling

In [3]:
!pip install kagglehub imbalanced-learn umap-learn -q

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, WeightedRandomSampler
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.metrics import classification_report, confusion_matrix, f1_score
from sklearn.metrics import balanced_accuracy_score, roc_auc_score
from imblearn.over_sampling import SMOTE

import kagglehub


In [4]:
# Chest X-Ray Dataset
chest_path = kagglehub.dataset_download("paultimothymooney/chest-xray-pneumonia")

# Flowers Dataset
flower_path = kagglehub.dataset_download("alxmamaev/flowers-recognition")

print("Chest Dataset:", chest_path)
print("Flower Dataset:", flower_path)


Using Colab cache for faster access to the 'chest-xray-pneumonia' dataset.


100%|██████████| 225M/225M [00:01<00:00, 136MB/s]

Extracting files...


Chest Dataset: /kaggle/input/chest-xray-pneumonia
Flower Dataset: /root/.cache/kagglehub/datasets/alxmamaev/flowers-recognition/versions/2


In [5]:
train_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

test_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])


In [6]:
flower_train = datasets.ImageFolder(
    root=f"{flower_path}/flowers",
    transform=train_transform
)

flower_loader = DataLoader(flower_train, batch_size=32, shuffle=True)

print("Flower Classes:", flower_train.classes)


Flower Classes: ['daisy', 'dandelion', 'rose', 'sunflower', 'tulip']


In [7]:
chest_train = datasets.ImageFolder(
    root=f"{chest_path}/chest_xray/train",
    transform=train_transform
)

chest_val = datasets.ImageFolder(
    root=f"{chest_path}/chest_xray/val",
    transform=test_transform
)

chest_test = datasets.ImageFolder(
    root=f"{chest_path}/chest_xray/test",
    transform=test_transform
)

print("Chest Classes:", chest_train.classes)


Chest Classes: ['NORMAL', 'PNEUMONIA']


SECTION B – DATA LEVEL TECHNIQUES

In [8]:
def create_weighted_sampler(dataset):
    targets = [label for _, label in dataset]
    class_counts = np.bincount(targets)
    class_weights = 1. / class_counts
    sample_weights = [class_weights[label] for label in targets]
    sampler = WeightedRandomSampler(sample_weights, len(sample_weights))
    return sampler

flower_sampler = create_weighted_sampler(flower_train)
chest_sampler = create_weighted_sampler(chest_train)

flower_loader_os = DataLoader(flower_train, batch_size=32, sampler=flower_sampler)
chest_loader_os = DataLoader(chest_train, batch_size=32, sampler=chest_sampler)


In [9]:
def undersample_dataset(dataset):
    targets = np.array([label for _, label in dataset])
    min_count = min(np.bincount(targets))
    indices = []

    for cls in np.unique(targets):
        cls_idx = np.where(targets == cls)[0]
        np.random.shuffle(cls_idx)
        indices.extend(cls_idx[:min_count])

    return torch.utils.data.Subset(dataset, indices)

flower_under = undersample_dataset(flower_train)
chest_under = undersample_dataset(chest_train)

flower_loader_us = DataLoader(flower_under, batch_size=32, shuffle=True)
chest_loader_us = DataLoader(chest_under, batch_size=32, shuffle=True)


In [10]:
def apply_smote(model, loader):
    model.eval()
    features = []
    labels = []

    with torch.no_grad():
        for images, target in loader:
            images = images.cuda() if torch.cuda.is_available() else images
            feat = model.features(images)
            feat = torch.flatten(feat, 1)
            features.append(feat.cpu().numpy())
            labels.append(target.numpy())

    X = np.vstack(features)
    y = np.hstack(labels)

    sm = SMOTE()
    X_res, y_res = sm.fit_resample(X, y)

    return X_res, y_res


ALGORITHM LEVEL TECHNIQUES

In [11]:
def get_class_weights(dataset):
    targets = [label for _, label in dataset]
    counts = np.bincount(targets)
    weights = 1.0 / torch.tensor(counts, dtype=torch.float)
    return weights

flower_weights = get_class_weights(flower_train)
chest_weights = get_class_weights(chest_train)

criterion_flower_weighted = nn.CrossEntropyLoss(weight=flower_weights)
criterion_chest_weighted = nn.CrossEntropyLoss(weight=chest_weights)


In [12]:
class CostSensitiveLoss(nn.Module):
    def __init__(self, class_weights):
        super().__init__()
        self.weights = class_weights

    def forward(self, outputs, targets):
        ce = nn.CrossEntropyLoss(reduction='none')(outputs, targets)
        weighted = ce * self.weights[targets]
        return weighted.mean()


In [13]:
def evaluate_with_threshold(model, loader, threshold=0.4):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for images, labels in loader:
            images = images.cuda() if torch.cuda.is_available() else images
            outputs = torch.softmax(model(images), dim=1)
            preds = (outputs[:,1] > threshold).int()
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.numpy())

    return classification_report(all_labels, all_preds)


EVALUATION

In [14]:
def evaluate_model(model, loader):
    model.eval()
    preds = []
    labels = []

    with torch.no_grad():
        for images, target in loader:
            images = images.cuda() if torch.cuda.is_available() else images
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            preds.extend(predicted.cpu().numpy())
            labels.extend(target.numpy())

    print(classification_report(labels, preds))
    print("Balanced Accuracy:", balanced_accuracy_score(labels, preds))
    print("Macro F1:", f1_score(labels, preds, average='macro'))


In [15]:
!pip install torchmetrics thop -q

import time
import seaborn as sns
from sklearn.metrics import confusion_matrix, roc_auc_score, precision_recall_curve, auc
from sklearn.metrics import classification_report
from torchmetrics.functional import accuracy
from thop import profile


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 23.2 MB/s eta 0:00:00


In [16]:
import torchvision.models as models

def get_resnet(num_classes):
    model = models.resnet18(pretrained=True)
    model.fc = nn.Linear(model.fc.in_features, num_classes)
    return model

def get_efficientnet(num_classes):
    model = models.efficientnet_b0(pretrained=True)
    model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes)
    return model


In [17]:
def train_model(model, train_loader, val_loader, criterion, optimizer, epochs=5):
    model.to(device)
    train_losses = []

    for epoch in range(epochs):
        model.train()
        running_loss = 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        train_losses.append(running_loss/len(train_loader))
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {train_losses[-1]:.4f}")

    return train_losses


In [18]:
def evaluate_model(model, loader, num_classes):
    model.eval()
    all_preds = []
    all_labels = []
    all_probs = []

    with torch.no_grad():
        for images, labels in loader:
            images = images.to(device)
            outputs = model(images)
            probs = torch.softmax(outputs, dim=1)

            _, preds = torch.max(outputs, 1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.numpy())
            all_probs.extend(probs.cpu().numpy())

    # Accuracy
    acc = np.mean(np.array(all_preds) == np.array(all_labels))

    # Top-3 Accuracy (for flowers)
    if num_classes > 2:
        top3 = accuracy(torch.tensor(all_probs),
                        torch.tensor(all_labels),
                        task="multiclass",
                        num_classes=num_classes,
                        top_k=3)
    else:
        top3 = None

    # F1 and Precision/Recall
    report = classification_report(all_labels, all_preds, output_dict=True)
    macro_f1 = report['macro avg']['f1-score']
    balanced_acc = balanced_accuracy_score(all_labels, all_preds)

    # Confusion Matrix
    cm = confusion_matrix(all_labels, all_preds)

    return acc, top3, macro_f1, balanced_acc, cm, all_probs, all_labels


In [19]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


Using device: cuda


In [20]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

num_classes_flower = len(flower_train.classes)

resnet_flower = get_resnet(num_classes_flower)
efficient_flower = get_efficientnet(num_classes_flower)

criterion = nn.CrossEntropyLoss()
optimizer_resnet = optim.Adam(resnet_flower.parameters(), lr=0.001)
optimizer_eff = optim.Adam(efficient_flower.parameters(), lr=0.001)

print("Training ResNet on Flowers")
train_model(resnet_flower, flower_loader_os, flower_loader_os, criterion, optimizer_resnet)

print("Training EfficientNet on Flowers")
train_model(efficient_flower, flower_loader_os, flower_loader_os, criterion, optimizer_eff)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
100%|██████████| 44.7M/44.7M [00:00<00:00, 178MB/s]
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weight

Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-7f5810bc.pth


100%|██████████| 20.5M/20.5M [00:00<00:00, 136MB/s]


Training ResNet on Flowers
Epoch [1/5], Loss: 0.6722
Epoch [2/5], Loss: 0.4964
Epoch [3/5], Loss: 0.4067
Epoch [4/5], Loss: 0.3484
Epoch [5/5], Loss: 0.3116
Training EfficientNet on Flowers
Epoch [1/5], Loss: 0.4390
Epoch [2/5], Loss: 0.2398
Epoch [3/5], Loss: 0.2172
Epoch [4/5], Loss: 0.1781
Epoch [5/5], Loss: 0.1597


[0.4390007338038197,
 0.23982654048336877,
 0.21716798877826443,
 0.1781011524023833,
 0.1596528406565388]

In [21]:
torch.save(efficient_flower.state_dict(),
           f"{save_path}/efficientnet_flower.pth")

torch.save(resnet_flower.state_dict(),
           f"{save_path}/resnet_flower.pth")

print("Models saved successfully!")


Models saved successfully!


In [22]:
num_classes_chest = len(chest_train.classes)

resnet_chest = get_resnet(num_classes_chest)
efficient_chest = get_efficientnet(num_classes_chest)

criterion = nn.CrossEntropyLoss()

optimizer_resnet = optim.Adam(resnet_chest.parameters(), lr=0.001)
optimizer_eff = optim.Adam(efficient_chest.parameters(), lr=0.001)

print("Training ResNet on Chest")
train_model(resnet_chest, chest_loader_os, chest_loader_os, criterion, optimizer_resnet)

print("Training EfficientNet on Chest")
train_model(efficient_chest, chest_loader_os, chest_loader_os, criterion, optimizer_eff)


Training ResNet on Chest
Epoch [1/5], Loss: 0.1493
Epoch [2/5], Loss: 0.0845
Epoch [3/5], Loss: 0.0797
Epoch [4/5], Loss: 0.0646
Epoch [5/5], Loss: 0.0668
Training EfficientNet on Chest
Epoch [1/5], Loss: 0.1440
Epoch [2/5], Loss: 0.0695
Epoch [3/5], Loss: 0.0591
Epoch [4/5], Loss: 0.0426
Epoch [5/5], Loss: 0.0498


[0.1439617920918333,
 0.06954269916207695,
 0.05907659520369389,
 0.0425636335551036,
 0.04978617959528336]

In [23]:
torch.save(efficient_chest.state_dict(),
           f"{save_path}/efficientnet_chest.pth")

torch.save(resnet_chest.state_dict(),
           f"{save_path}/resnet_chest.pth")

print("Models saved successfully!")


Models saved successfully!


In [24]:
acc_r, top3_r, f1_r, bal_r, cm_r, probs_r, labels_r = evaluate_model(
    resnet_flower, flower_loader, num_classes_flower)

acc_e, top3_e, f1_e, bal_e, cm_e, probs_e, labels_e = evaluate_model(
    efficient_flower, flower_loader, num_classes_flower)

print("ResNet Flower Accuracy:", acc_r)
print("EfficientNet Flower Accuracy:", acc_e)


/tmp/ipython-input-1306553613.py:24: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:253.)
  top3 = accuracy(torch.tensor(all_probs),


ResNet Flower Accuracy: 0.87607134584202
EfficientNet Flower Accuracy: 0.9722029186935371
